# Needleman-Wunsch Alignment Algorithm - Naive version

In [2]:
import csv

## Blosum 50

In [7]:
class Blosum50:

    def __init__(self, header, matrix):
        self._header = header
        self._matrix = matrix

    def _get_symbol_index(self, symbol: str) -> int:
        if symbol in self._header:
            return self._header.index(symbol)
        else:
            return self._header.index("*")

    def get(self, first: str, second: str) -> int:
        first_index = self._get_symbol_index(first)
        second_index = self._get_symbol_index(second)

        return self._matrix[first_index][second_index]


    @classmethod
    def load(cls, filename: str) -> 'Blosum50':
        matrix = []
        header = None

        with open(filename, "r") as handle:
            reader = csv.reader(handle)

            header = next(reader, None)
            if header is None:
                raise ValueError("Missing header")

            for row in reader:
                matrix.append(
                    list(
                        map(
                            int,
                            row
                        )
                    )
                )

        return cls(header, matrix)


blosum50 = Blosum50.load("needleman-wunsch/blosum50.csv")
blosum50.get("P", "H")

-2

## Needleman-Wunsch

In [110]:
class NeedlemanWunsch:

    def __init__(self, a: str, b: str, d: float, blosum50: Blosum50):
        self._a = a
        self._b = b
        self._d = d
        self._blosum50 = blosum50

    def build_matrix(self):
        self._matrix = [
            [(0, 0) for _ in range(len(self._a) + 1)]
            for _ in range(len(self._b) + 1)
        ]

        for row in range(1, len(self._b) + 1):
            self._matrix[row][0] = (0, -row * self._d)

        for column in range(1, len(self._a) + 1):
            self._matrix[0][column] = (1, -column * self._d)

        for row in range(1, len(self._b) + 1):
            for column in range(1, len(self._a) + 1):
                if row == column == 0:
                    continue

                self._matrix[row][column] = argmax(
                    self._matrix[row - 1][column][1] - self._d,
                    self._matrix[row][column - 1][1] - self._d,
                    self._matrix[row - 1][column - 1][1] + self._blosum50.get(
                       self._a[column - 1], self._b[row - 1]
                    )
                )

        for line in self._matrix:
            print(line)

    def align(self):
        first = []
        second = []
        row = len(self._b)
        column = len(self._a)
        print("VALUE: ", self._matrix[row][column][1])

        while row != 0 or column != 0:
            direction, _ = self._matrix[row][column]
            
            if direction == 0:
                row -= 1
                first.append("-")
                second.append(self._b[row])
            elif direction == 1:
                column -= 1
                first.append(self._a[column])
                second.append("-")
            else:
                column -= 1
                row -= 1
                first.append(self._a[column])
                second.append(self._b[row])

        first_join = "".join(reversed(first))
        second_join = "".join(reversed(second))          
        return first_join, second_join

example = NeedlemanWunsch("HEAGAWGHEE", "PAWHEAE", 8, blosum50)
example.build_matrix()
example.align()

[(0, 0), (1, -8), (1, -16), (1, -24), (1, -32), (1, -40), (1, -48), (1, -56), (1, -64), (1, -72), (1, -80)]
[(0, -8), (2, -2), (2, -9), (1, -17), (1, -25), (1, -33), (1, -41), (1, -49), (1, -57), (1, -65), (1, -73)]
[(0, -16), (0, -10), (2, -3), (2, -4), (1, -12), (1, -20), (1, -28), (1, -36), (1, -44), (1, -52), (1, -60)]
[(0, -24), (0, -18), (0, -11), (2, -6), (2, -7), (1, -15), (2, -5), (1, -13), (1, -21), (1, -29), (1, -37)]
[(0, -32), (2, -14), (2, -18), (2, -13), (2, -8), (2, -9), (0, -13), (2, -7), (2, -3), (1, -11), (1, -19)]
[(0, -40), (0, -22), (2, -8), (1, -16), (0, -16), (2, -9), (2, -12), (0, -15), (2, -7), (2, 3), (1, -5)]
[(0, -48), (0, -30), (0, -16), (2, -3), (1, -11), (2, -11), (2, -12), (2, -12), (0, -15), (0, -5), (2, 2)]
[(0, -56), (0, -38), (0, -24), (0, -11), (2, -6), (2, -12), (2, -14), (2, -15), (2, -12), (2, -9), (2, 1)]
VALUE:  1


('HEAGAWGHE-E', '-PA--W-HEAE')